<a href="https://colab.research.google.com/github/Multik838/-Data-analysis-algorithms/blob/main/%D0%A3%D1%80%D0%BE%D0%BA_5_HW_wf_udf_joins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 46.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=0c905f1d522506072eb3f894881a41eb5b356b6ff3bd58621eca796ab36916fc
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_2")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

# Самостоятельная работа к уроку 3
На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания.

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

In [4]:
from google.colab import drive
drive.mount('/content/drive')

raw_sales = spark.read.format('csv').option('header', 'True').load('/content/raw_sales.csv') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
raw_sales.show(1)

+-------------------+--------+------+------------+--------+
|           datesold|postcode| price|propertyType|bedrooms|
+-------------------+--------+------+------------+--------+
|2007-02-07 00:00:00|    2607|525000|       house|       4|
+-------------------+--------+------+------------+--------+
only showing top 1 row



In [7]:
#Импортируем необходимые библиотеки
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import *

## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего (1 балл)


In [ ]:
windSpec = Window()\
    .partitionBy('Country')\
    .orderBy('Amount') \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow - 1)

df.withColumn('lag', F.last('Product').over(windSpec)).show()

+-------+------+-------+-------+
|Product|Amount|Country|    lag|
+-------+------+-------+-------+
| Banana|  2000| Canada|   null|
|Carrots|  2000| Canada| Banana|
| Banana|   400|  China|   null|
|Carrots|  1200|  China| Banana|
|  Beans|  1500|  China|Carrots|
| Orange|  4000|  China|  Beans|
|  Beans|  2000| Mexico|   null|
| Banana|  1000|    USA|   null|
|Carrots|  1500|    USA| Banana|
|  Beans|  1600|    USA|Carrots|
| Orange|  2000|    USA|  Beans|
| Orange|  2000|    USA| Orange|
+-------+------+-------+-------+



## Задание 2
Найдите среднюю цену жилья для каждого года и приджойните эти данные к таблице из задания 1. (2 балла)


*(left join on a.year(date of sale) = b.year, где a - таблица из первого задания, а b таблица после группировки)*

## Задание 3
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)). (2 балла)